# ML-Net for silency prediction
##### Mount drive
In testing enviroment learning data were stored on Google drive.

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

##### Import needed libraries
To make things work ensure that you have this libraries installed.

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import math
from keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints
import keras.backend as K

##### Configuration parameters

In [ ]:
BATCH = 10 # batch size

IMG_HEIGHT = 480 # input image width
IMG_WIDTH = 640 # input image height

MAP_HEIGHT = int(math.ceil(IMG_HEIGHT / 8)) # extracted feature map height
MAP_WIDTH = int(math.ceil(IMG_WIDTH / 8)) # extracted feature map width 

EPOCHS = 40 # number of epochs

TRAIN_IMG_PATH = "/content/drive/MyDrive/salicon/images/train" # path to training images
VAL_IMG_PATH = "/content/drive/MyDrive/salicon/images/val" # path to validation images
TRAIN_MAP_PATH = "/content/drive/MyDrive/salicon/maps/train" # path to training maps
VAL_MAP_PATH = "/content/drive/MyDrive/salicon/maps/val" # path to validation maps

TRAIN_SIZE = 1000 # number of training samples
VAL_SIZE = 500 # number of validation samples

##### Eltwise product layer
Custom layer responsible for learning prior bias.

In [ ]:
class EltWiseProduct(Layer):
    def __init__(self, downsampling_factor=10, init='glorot_uniform', activation='linear',
                 weights=None, W_regularizer=None, activity_regularizer=None,
                 W_constraint=None, input_dim=None, **kwargs):
        self.downsampling_factor = downsampling_factor
        self.init = initializers.get(init)
        self.activation = activation
        self.W_regularizer = regularizers.get(W_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.input_dim = input_dim

        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)

        self.input_spec = InputSpec(ndim=4)
        super(EltWiseProduct, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_shape = (MAP_HEIGHT, MAP_WIDTH, 1)   # Adjusted weight shape for broadcasting
        self.W = self.add_weight(shape=self.W_shape,
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None,) + input_shape[1:])
        self.built = True

    def call(self, x, mask=None):
        output = x * tf.expand_dims(1 + self.W, 0)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {
            'downsampling_factor': self.downsampling_factor,
            'init': initializers.serialize(self.init),
            'activation': self.activation,
            'W_regularizer': regularizers.serialize(self.W_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'W_constraint': constraints.serialize(self.W_constraint),
            'input_dim': self.input_dim
        }
        base_config = super(EltWiseProduct, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


##### Model

Pre-trained feature extraction network 

In [ ]:
vgg19 = tf.keras.applications.vgg19.VGG19(input_shape=(480, 640, 3), include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


Actual model

In [ ]:
input = tf.keras.Input(shape=(480, 640, 3))

feature_extraction = tf.keras.Sequential(layers=vgg19.layers[:11])(input)
conv3_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding="same")(feature_extraction)
feature_extraction_1 = tf.keras.Sequential(layers=vgg19.layers[12:16])(conv3_pool)
conv4_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding="same")(feature_extraction_1)
more_features = tf.keras.Sequential(layers=vgg19.layers[17:20])(conv4_pool)
conv5_4 = tf.keras.layers.Conv2D(512, 3, weights=vgg19.layers[20].get_weights(), activation='relu', padding='same')(more_features)

concatenated = tf.keras.layers.concatenate([conv3_pool, conv4_pool, conv5_4], axis=-1)
dropout = tf.keras.layers.Dropout(0.5)(concatenated)

int_conv = tf.keras.layers.Conv2D(64, 3, kernel_initializer='glorot_normal', activation='relu', padding='same')(dropout)
pre_final_conv = tf.keras.layers.Conv2D(1, 1, kernel_initializer='glorot_normal', activation='relu')(int_conv)

downsampling_factor_net = 8
downsampling_factor_product = 10
rows_elt = math.ceil(IMG_HEIGHT / downsampling_factor_net) // downsampling_factor_product
cols_elt = math.ceil(IMG_WIDTH / downsampling_factor_net) // downsampling_factor_product
eltprod = EltWiseProduct(init='zero', W_regularizer=tf.keras.regularizers.L2(1/(rows_elt*cols_elt)))(pre_final_conv)

pre_output = tf.keras.layers.Activation('relu')(eltprod)
output = tf.keras.layers.Resizing(IMG_HEIGHT, IMG_WIDTH, interpolation='bicubic')(pre_output)

better_model = tf.keras.Model(inputs=input, outputs=output, name="mlnet-vgg19")

better_model.summary()

Model: "mlnet-vgg19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 480, 640, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 120, 160, 256)        2325568   ['input_2[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 60, 80, 256)          0         ['sequential[0][0]']          
 D)                                                                                               
                                                                                                  
 sequential_1 (Sequential)   (None, 60, 80, 512)          8259584   ['max_pooling2d[0][0

##### Custom loss function

In [ ]:
def loss(y_true, y_pred):
  max_y = K.max(K.max(y_pred, axis=1), axis=1)  # Adjust axis values
  max_y = K.repeat_elements(K.expand_dims(K.repeat_elements(K.expand_dims(max_y, axis=-1), IMG_HEIGHT, axis=-1)), IMG_WIDTH, axis=-1)
  return K.mean(K.square((y_pred / max_y) - y_true) / (1.1 - y_true))

##### Compiling model

In [ ]:
sgd = tf.keras.optimizers.experimental.SGD(learning_rate=1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
better_model.compile(optimizer=sgd, loss=loss)

##### Data generator

In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, labels, batch_size, size):
      datas = [f for f in os.listdir(data)]
      labeles = [f for f in os.listdir(labels)]
      datas.sort()
      labeles.sort()
      self.images = [data + "/" + f for f in datas[:size]]
      self.maps = [labels + "/" + f for f in labeles[:size]]

      self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        x = []
        y = []
        for f in self.images[start:end]:
          image = cv2.imread(f)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image = tf.keras.applications.vgg16.preprocess_input(image)
          x.append(image)

        for f in self.maps[start:end]:
          image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
          y.append(image.astype('float32') / 255)

        # Implement any data preprocessing here if needed
        return np.array(x), np.array(y)

In [ ]:
train_generator = CustomDataGenerator(TRAIN_IMG_PATH, TRAIN_MAP_PATH, BATCH, TRAIN_SIZE)
validation_generator = CustomDataGenerator(VAL_IMG_PATH, VAL_MAP_PATH, BATCH, VAL_SIZE)

##### Learning model

In [ ]:
better_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH, validation_data=validation_generator, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5), tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/salicon/mlnet-bigger_prior-{epoch:02d}-{val_loss:.2f}.hdf5',save_best_only=True,verbose=1)])

Epoch 1/40
100/100 [==============================] - ETA: 0s - loss: 0.0551
Epoch 1: val_loss improved from inf to 0.05152, saving model to /content/drive/MyDrive/salicon/mlnet-bigger_prior-01-0.05.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 766s 7s/step - loss: 0.0551 - val_loss: 0.0515
Epoch 2/40
100/100 [==============================] - ETA: 0s - loss: 0.0490
Epoch 2: val_loss improved from 0.05152 to 0.04609, saving model to /content/drive/MyDrive/salicon/mlnet-bigger_prior-02-0.05.hdf5
100/100 [==============================] - 215s 2s/step - loss: 0.0490 - val_loss: 0.0461
Epoch 3/40
100/100 [==============================] - ETA: 0s - loss: 0.0465
Epoch 3: val_loss improved from 0.04609 to 0.04469, saving model to /content/drive/MyDrive/salicon/mlnet-bigger_prior-03-0.04.hdf5
100/100 [==============================] - 215s 2s/step - loss: 0.0465 - val_loss: 0.0447
Epoch 4/40
100/100 [==============================] - ETA: 0s - loss: 0.0446
Epoch 4: val_loss improved from 0.04469 to 0.04191, saving model to /content/drive/MyDrive/salicon/mlnet-bigger_prior-04-0.04.hdf5
100/100 [==============================] - 215s 2s/step - loss: 0.0446 - val_loss: 0.0419
Epoch 5/40
100/1